In [ ]:
# AML Transaction Monitoring – SQL Analytics (Nivel 1)

Este notebook analiza patrones básicos de riesgo utilizando consultas SQL ejecutadas en el sistema `aml_system`.


In [ ]:
## 1. Introducción

Dataset simulado con:
- 5 clientes (diferentes perfiles de riesgo)
- 12 transacciones
- Comportamientos de interés AML:
  - Envíos a países de alto riesgo (Panamá, UAE)
  - Montos altos >5000 USD
  - Structuring: montos repetidos menores a 500 USD
  - Actividad internacional inconsistente con perfil

Objetivo del notebook:
Identificar señales de alerta (Red Flags) utilizando consultas SQL ejecutadas desde MySQL.


In [ ]:
## 2. Red Flag: High-Risk Transactions (>5000 USD, Panamá/UAE)

Consulta SQL utilizada:

```sql
SELECT 
    T.id_transaccion,
    C.nombre_cliente,
    T.monto_usd,
    T.pais_destino,
    T.canal,
    T.timestamp_creacion
FROM Transacciones T
INNER JOIN Clientes C ON T.cliente_id = C.id_cliente
WHERE 
    T.monto_usd >= 5000
    AND T.pais_destino IN ('Panama', 'UAE')
ORDER BY T.monto_usd DESC;


In [ ]:
Resultado esperado:

Fatima Al-Said: envíos repetidos y de alto valor a Panamá.

Ana Rodríguez: operaciones de +5000 USD repetidas hacia Panamá.

Análisis AML:

Estos patrones son consistentes con:

Uso de jurisdicciones de alto riesgo

Montos inusuales vs perfil

Potencial Layering / Transferencias sospechosas

In [ ]:

---

### 📗 **CELDA 4 — Structuring (<500 repetido)**
```markdown
## 3. Red Flag: Structuring (<500 USD repetido)

Consulta:

```sql
SELECT 
    cliente_id,
    COUNT(*) AS frecuencia,
    SUM(monto_usd) AS monto_acumulado
FROM Transacciones
WHERE monto_usd < 500
GROUP BY cliente_id
HAVING COUNT(*) >= 3;


In [ ]:
Resultado esperado:

Cliente 4 (Carlos Gómez) aparece con 3 transacciones de 499 USD.

Análisis AML:

Esto es un patrón típico de estructuración de efectivo, donde el cliente divide montos para evitar alertas automáticas.

In [ ]:

---

### 📗 **CELDA 5 — Actividad internacional sospechosa**
```markdown
## 4. Red Flag: Actividad internacional inconsistente

Consulta:

```sql
SELECT
    C.nombre_cliente,
    COUNT(*) AS tx_internacionales
FROM Transacciones T
INNER JOIN Clientes C ON T.cliente_id = C.id_cliente
WHERE T.pais_destino <> C.pais_residencia
GROUP BY C.id_cliente
HAVING tx_internacionales >= 2;


In [ ]:
Análisis:

Clientes con actividad fuera de su jurisdicción pueden ser señales de:

Layering

Smurfing internacional

Envíos sin justificación económica clara

In [ ]:

---

### 📗 **CELDA 6 — Conclusión**
```markdown
## 5. Conclusión Final (Nivel 1 AML Analytics)

### Clientes con mayor riesgo:
- **Fatima Al-Said** → Alto riesgo + montos elevados + Panamá
- **Ana Rodríguez** → País de residencia de alto riesgo + transferencias grandes
- **Carlos Gómez** → Structuring evidente (<500 USD repetidos)

### Siguiente paso para el Día 4:
- Crear dashboard AML con Excel/PowerBI.
- Iniciar Data Cleaning y EDA con Python.
